In [9]:
# 1. Setup and initialization
import weaviate
from weaviate.classes.init import Auth
from weaviate.classes.query import MetadataQuery
from langsmith import traceable
from langsmith.wrappers import wrap_openai

from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

# Initialize clients
wcd_url = os.environ["WCD_URL"]
wcd_api_key = os.environ["WCD_API_KEY"]

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,                                    
    auth_credentials=Auth.api_key(wcd_api_key),
)
openai_client = wrap_openai(OpenAI(api_key=os.getenv('OPENAI_API_KEY')))


In [10]:
# 2. Define the query
query = "How do I report a bug with my Tesla Cybertruck?"

# 2a. Determine if the query requires retrieval from the vector database

@traceable
def check_rag(query):
    retrieval_test_response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": f"""You are a helpful assistant that determines whether a given user prompt is related to Tesla Cybertrucks
            or if it is a more general prompt. If it is a general prompt, you should respond with a helpful response. If it is related to Tesla Cybertrucks,
            you should not answer the question, but simply rephrase the prompt to be a complete sentence and return a response in the format:
            <SYSTEM>[rephrased prompt]</SYSTEM>"""},
            {"role": "user", "content": query}
        ],
        temperature=0
    )
    return retrieval_test_response.choices[0].message.content

answer = check_rag(query)

import re
matches = re.findall(r"<SYSTEM>(.*?)</SYSTEM>", answer)
if len(matches) > 0:
    query = matches[0]
    do_rag = True
else:
    prompt = query
    do_rag = False 


if do_rag:
    # 3. Get embedding for the query
    response = openai_client.embeddings.create(
        model="text-embedding-3-large",
        input=query
    )
    query_embedding = response.data[0].embedding


In [11]:
if do_rag:
    # 4. Search for similar texts in Weaviate
    collection_name = "James"

    collection = client.collections.get(collection_name)
    similar_texts = collection.query.near_vector(
        near_vector=query_embedding,
        limit=3,
        return_properties=["text"],
        return_metadata=MetadataQuery(distance=True)
    )

    print("\nRetrieved contexts:")
    for i, doc in enumerate(similar_texts.objects, 1):
        print(f"\n{i}. Distance: {doc.metadata.distance}")
        print(f"Text: {doc.properties['text']}")
        print("\n\n")



Retrieved contexts:

1. Distance: 0.32153111696243286
Text: Contacting Tesla

For detailed information about your Cybertruck, go to http://www.tesla.com and log on to your Tesla account or sign up to get an account.

If you have any questions or concerns about your Cybertruck, in the United States, Canada or Puerto Rico, call 1-877-79TESLA (1-877-798-3752) and in Mexico, call 1-800-228-8145.

NOTE: You can also use voice commands to provide feedback to Tesla. Say "Report", "Feedback", or "Bug report" followed by brief comments. Cybertruck takes a snapshot of its systems, including your current location, vehicle diagnostic data, and screen captures of the touchscreen. Tesla periodically reviews these notes and uses them to continue improving Cybertruck.




2. Distance: 0.36803579330444336
Text: Reporting Safety Defects




3. Distance: 0.37044668197631836
Text: Reporting Safety Defects - Canada

If you believe that your Cybertruck has a defect which could cause a crash or could cause 

In [12]:
if do_rag:
    # 5. Prepare context for GPT
    context_str = "\n\n---\n\n".join([doc.properties["text"] for doc in similar_texts.objects])

    prompt = f"""Answer the question using ONLY the information provided in the context below. 
    Do not add any general knowledge or information not contained in the context."

    Context:
    {context_str}

    Question: {query}

    Answer:"""

    # 6. Generate answer using GPT-4
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )

    answer = response.choices[0].message.content

# 7. Print results
print(f"Prompt: {prompt} \n\n----------------------------------\n\n")
print(f"Question: {query}\n\nAnswer: {answer}")


Prompt: Answer the question using ONLY the information provided in the context below. 
    Do not add any general knowledge or information not contained in the context."

    Context:
    Contacting Tesla

For detailed information about your Cybertruck, go to http://www.tesla.com and log on to your Tesla account or sign up to get an account.

If you have any questions or concerns about your Cybertruck, in the United States, Canada or Puerto Rico, call 1-877-79TESLA (1-877-798-3752) and in Mexico, call 1-800-228-8145.

NOTE: You can also use voice commands to provide feedback to Tesla. Say "Report", "Feedback", or "Bug report" followed by brief comments. Cybertruck takes a snapshot of its systems, including your current location, vehicle diagnostic data, and screen captures of the touchscreen. Tesla periodically reviews these notes and uses them to continue improving Cybertruck.

---

Reporting Safety Defects

---

Reporting Safety Defects - Canada

If you believe that your Cybertruck h